In [8]:
import torch
from torch import nn
from torch.utils.data import TensorDataset,DataLoader

class RNN(nn.Module):
    def __init__(self,vocab,emb,pad,out,hid):
        super().__init__()
        self.hidden_size=hid
        self.emb=nn.Embedding(vocab,emb,padding_idx=pad)
        self.rnn=nn.RNN(emb,hid,batch_first=True)
        self.linear=nn.Linear(hid,out)
        self.softmax=nn.Softmax(dim=1)
        
    def forward(self,x,h=None):
        x=self.emb(x)
        y,h=self.rnn(x,h)
        y=y[:,-1,:]
        y=self.linear(y)
        y=self.softmax(y)
        return y
    
def count(file):
    all_word=[]
    with open(file,"r",encoding="utf-8") as tf:
        tags=[]
        tag_num={"b":0,"t":1,"e":2,"m":3}
        for line in tf:
            line.strip();
            tag,sen=line.split("\t")
            tags.append(tag_num[tag])
            words=sen.strip().split(" ")
            for word in words:
                all_word.append(word)
        freq=collections.Counter(all_word)
        freq_sort=sorted(freq.items(),key=lambda x:x[1],reverse=True)
        #print(freq_sort)
        word_number={w:i+1 for i,(w,f) in enumerate(freq_sort) if f>=2}
        return word_number,tags

def id_v(file,word_id):
    with open(file,"r",encoding="utf-8") as tf:
        ids=[]
        for line in tf:
            line.strip();
            tag,sen=line.split("\t")
            words=sen.strip().split(" ")
            sent_ids=[]
            for word in words:
                if word in word_id:
                    sent_ids.append(word_id[word])
                else:
                    sent_ids.append(0)
            ids.append(sent_ids)
        return ids

def list2tensor(data,max_len,pad):
    new_list=[]
    for d in data:
        if len(d)>max_len:
            d=d[:max_len]
        else:
            d+=[pad]*(max_len-len(d))
        new_list.append(d)
    return torch.tensor(new_list)

word_id,Y_train=count("train.feature.txt")
word_id_valid,Y_valid=count("valid.feature.txt")
max_len=10
vocab=len(word_id)+1
emb=300
pad=len(word_id)
hid=50
out=4
X_train=id_v("train.feature.txt",word_id)
X_train=list2tensor(X_train,max_len,pad)
Y_train=torch.tensor(Y_train)
model=RNN(vocab,emb,pad,out,hid)
dataset_train=TensorDataset(X_train,Y_train)
for i in range(5):
    X=dataset_train[i][0]
    print(torch.softmax(model(X.unsqueeze(0)),dim=-1))

tensor([[0.2429, 0.2777, 0.2522, 0.2272]], grad_fn=<SoftmaxBackward>)
tensor([[0.2485, 0.2299, 0.2431, 0.2785]], grad_fn=<SoftmaxBackward>)
tensor([[0.2202, 0.2657, 0.2724, 0.2416]], grad_fn=<SoftmaxBackward>)
tensor([[0.2474, 0.2514, 0.2506, 0.2506]], grad_fn=<SoftmaxBackward>)
tensor([[0.2560, 0.2569, 0.2382, 0.2489]], grad_fn=<SoftmaxBackward>)
